<a href="https://colab.research.google.com/github/Nicordaro/ml4iot_hw3/blob/main/small_model_stft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tensorflow_model_optimization

     |████████████████████████████████| 174kB 8.1MB/s 


In [2]:
import os
# Clone github repository with data
# if os.path.isdir('./Project_MLDL'):
!rm -rf Project_MLDL
DATA_DIR = './ml4iot_hw3'
if not os.path.isdir(DATA_DIR):
  !git clone https://github.com/Nicordaro/ml4iot_hw3

Cloning into 'ml4iot_hw3'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 67 (delta 39), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (67/67), done.


In [3]:
import argparse
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_model_optimization as tfmot
import zlib
import shutil

seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

zip_path = tf.keras.utils.get_file(
                origin = 'http://storage.googleapis.com/download.tensorflow.org/data/mini_speech_commands.zip',
                fname = 'mini_speech_commands.zip',
                extract = True,
                cache_dir = '.',
                cache_subdir = 'data')

data_dir = os.path.join('.','data', 'mini_speech_commands')

def filenames(path) :
    with open(path, "rb") as fp:
        arr = np.array(fp.read().splitlines())
    return arr
    
train_files = filenames(DATA_DIR+"/kws_train_split.txt")
val_files = filenames(DATA_DIR+"/kws_val_split.txt")
test_files = filenames(DATA_DIR+"/kws_test_split.txt")

LABELS = np.array(tf.io.gfile.listdir(str(data_dir)))
LABELS = LABELS[LABELS != 'README.md']

182083584/182082353 [==============================] - 1s 0us/step


In [4]:
class SignalGenerator:
    def __init__(self, labels, sampling_rate, frame_length, frame_step, num_mel_bins=None, lower_frequency=None,
                upper_frequency=None, num_coefficients=None, mfcc=False):
        
        self.labels = labels
        self.sampling_rate = sampling_rate 
        self.frame_length = frame_length  
        self.frame_step = frame_step       
        self.num_mel_bins = num_mel_bins   
        self.lower_frequency = lower_frequency  
        self.upper_frequency = upper_frequency  
        self.num_coefficients = num_coefficients  
        self.mfcc = mfcc  
        
        if mfcc is True:
            self.preprocess = self.preprocess_with_mfcc
        else :
            self.preprocess = self.preprocess_with_stft
            
            
    def read(self, file_path):
        parts = tf.strings.split(file_path, os.path.sep)
        label = parts[-2]  
        label_id = tf.argmax(label == self.labels)
        
        audio_binary = tf.io.read_file(file_path)
        audio,_ = tf.audio.decode_wav(audio_binary)
        audio = tf.squeeze(audio, axis=1)  
        audio = self.pad(audio)
        
        return audio, label_id
    
    def pad(self, audio):
        zero_padding = tf.zeros([self.sampling_rate] - tf.shape(audio), dtype=tf.float32)
        audio = tf.concat([audio, zero_padding], 0)
        audio.set_shape([self.sampling_rate])
        
        return audio
      
    def preprocess_with_stft(self, file_path):
        audio, label = self.read(file_path)
        
        stft = tf.signal.stft(audio, frame_length=self.frame_length, frame_step=self.frame_step, fft_length=self.frame_length)
        spectrogram = tf.abs(stft)
        
        if self.mfcc == False:
            spectrogram = tf.expand_dims(spectrogram, -1)
            spectrogram = tf.image.resize(spectrogram, [50,50])
            
        return spectrogram, label
    
    
    def preprocess_with_mfcc(self, file_path):
        
        spectrogram, label = self.preprocess_with_stft(file_path)
        
        num_spectrogram_bins = spectrogram.shape[-1]
        linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(self.num_mel_bins, num_spectrogram_bins,
                                                                            self.sampling_rate, 
                                                                            self.lower_frequency, self.upper_frequency)
        
        mel_spectrogram = tf.tensordot(spectrogram, linear_to_mel_weight_matrix, 1)
        log_mel_spectrogram = tf.math.log(mel_spectrogram + 1e-6)
        mfccs = tf.signal.mfccs_from_log_mel_spectrograms(log_mel_spectrogram)[..., :self.num_coefficients]
        mfccs = tf.expand_dims(mfccs, -1)
          
        return mfccs, label
        
    
    def make_ds(self, filenames, train):
        ds = tf.data.Dataset.from_tensor_slices(filenames)
        ds = ds.map(self.preprocess, num_parallel_calls=4)
        ds = ds.batch(32)
        ds = ds.cache()
        
        if train == True:
            ds = ds.shuffle(100, reshuffle_each_iteration=True)
            
        return ds

In [5]:
sampling_rate = 16000
frame_length = 256
frame_step = 128

In [19]:
generator = SignalGenerator(LABELS, sampling_rate, frame_length, frame_step)                          # STFT

train_ds = generator.make_ds(train_files, True)
val_ds = generator.make_ds(val_files, False)
test_ds = generator.make_ds(test_files, False)

In [20]:
def model_creation(name, alpha, mfcc=False):
    
    input_shape = (50,50)
    stride = [2,2]  #[2,2]
        
    if name == "cnn":
        model = keras.Sequential([
            keras.layers.Conv2D(filters = int(128*alpha), kernel_size=[3,3], strides=stride, use_bias=False, activation='relu'), 
            keras.layers.BatchNormalization(momentum=0.1),
            keras.layers.Conv2D(filters = int(128*alpha), kernel_size=[3,3], strides=[1,1], use_bias=False, activation='relu'), 
            keras.layers.BatchNormalization(momentum=0.1),
            keras.layers.Conv2D(filters = int(128*alpha), kernel_size=[3,3], strides=[1,1], use_bias=False, activation='relu'), 
            keras.layers.BatchNormalization(momentum=0.1),
            keras.layers.GlobalAveragePooling2D(),
            keras.layers.Dense(8)
        ])
    
    return model

In [21]:
alpha = 0.4
mfcc = False
number_of_clusters = 16
    
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.02)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy()
n_epochs = 20
model_name = 'cnn'

def scheduler(epoch, lr):
    if epoch == 10 or epoch == 17:
        return lr*0.1   
    return lr

In [22]:
# Callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', mode = 'min', min_delta = 0.005, patience = 5)

checkpoint = tf.keras.callbacks.ModelCheckpoint( './solutions Group16/small_model_stft', 
                                                 save_best_only = True,
                                                 save_weights_only = True,
                                                 monitor = 'val_loss',
                                                 mode = 'min')

schedule_callback = tf.keras.callbacks.LearningRateScheduler(schedule = scheduler)

In [ ]:
cnn = model_creation(model_name, alpha, mfcc = mfcc)

cnn.compile(optimizer = optimizer,
            loss = loss,              
            metrics = metric)

cnn.fit(train_ds, epochs = n_epochs, validation_data=val_ds, callbacks = [schedule_callback, checkpoint])

Epoch 1/20
200/200 [==============================] - 14s 40ms/step - loss: 1.7538 - sparse_categorical_accuracy: 0.3470 - val_loss: 1.1071 - val_sparse_categorical_accuracy: 0.5813
Epoch 2/20
200/200 [==============================] - 1s 5ms/step - loss: 0.9347 - sparse_categorical_accuracy: 0.6673 - val_loss: 0.7080 - val_sparse_categorical_accuracy: 0.7262
Epoch 3/20
200/200 [==============================] - 1s 4ms/step - loss: 0.6309 - sparse_categorical_accuracy: 0.7892 - val_loss: 0.4929 - val_sparse_categorical_accuracy: 0.8487
Epoch 4/20
200/200 [==============================] - 1s 4ms/step - loss: 0.4886 - sparse_categorical_accuracy: 0.8412 - val_loss: 0.5220 - val_sparse_categorical_accuracy: 0.8375
Epoch 5/20
200/200 [==============================] - 1s 4ms/step - loss: 0.4158 - sparse_categorical_accuracy: 0.8676 - val_loss: 0.4157 - val_sparse_categorical_accuracy: 0.8875
Epoch 6/20
200/200 [==============================] - 1s 4ms/step - loss: 0.3607 - sparse_categori

In [ ]:
cnn.load_weights('./solutions Group16/small_model_stft')
test_loss, test_accuracy = cnn.evaluate(test_ds, verbose=2)
test_accuracy

In [ ]:
# Weights clustering
cnn_clustered = tfmot.clustering.keras.cluster_weights(cnn, 
                                            number_of_clusters = number_of_clusters,
                                            cluster_centroids_init = tfmot.clustering.keras.CentroidInitialization.LINEAR)

cnn_clustered.compile(optimizer = optimizer,
            loss = loss,              
            metrics = metric)

cnn_clustered.fit(train_ds, epochs = n_epochs, validation_data=val_ds, callbacks = [checkpoint, schedule_callback, early_stopping])

In [ ]:
cnn_clustered.load_weights('./solutions Group16/small_model_stft')
test_loss, test_accuracy = cnn_clustered.evaluate(test_ds, verbose=2)
test_accuracy

In [ ]:
# Save the clustered model
cnn_clustered = tfmot.clustering.keras.strip_clustering(cnn_clustered)
cnn_clustered.save('./solutions Group16/small_model_stft')

In [ ]:
# Save the tflite
converter = tf.lite.TFLiteConverter.from_saved_model('./solutions Group16/small_model_stft')
    
tflite_model = converter.convert()

with open('small_model_stft.tflite.zlib', 'wb') as fp:
    fp.write(tflite_model)

In [ ]:
tflite_model = converter.convert()

with open('small_model_stft.tflite.zlib', 'wb') as fp:
    tflite_compressed = zlib.compress(tflite_model)
    fp.write(tflite_compressed)

In [ ]:
# Delete the directory with keras models and checkpoints
#shutil.rmtree('solutions Group16', ignore_errors=True)

In [ ]:
# Test the model
test_ds_tflite = test_ds.unbatch().batch(1)

def acc_inference(interpreter):
    
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details() 
    output_details = interpreter.get_output_details()
    
    acc = tf.keras.metrics.SparseCategoricalAccuracy()
    
    for (x,y) in test_ds_tflite:
        interpreter.set_tensor(input_details[0]["index"], x)
        interpreter.invoke()
        y_pred = interpreter.get_tensor(output_details[0]["index"])
        
        y_true = y
        acc.update_state(y_true, y_pred)
    
    return acc.result().numpy()

In [ ]:
with open('small_model_stft.tflite.zlib', 'rb') as fp:
    model_zip = zlib.decompress(fp.read())
    interpreter = tf.lite.Interpreter(model_content=model_zip)
    tflite_acc = acc_inference(interpreter)
tflite_acc, os.path.getsize('small_model_stft.tflite.zlib')